In [1]:
# Import libraries
import os

import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import LSTM, Dense

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
df_finbert = pd.read_csv('/kaggle/input/redditnews/RedditNews_FinBert.csv')
df_vader = pd.read_csv('/kaggle/input/redditnews/RedditNews_Vader.csv')

df_stock = pd.read_csv('/kaggle/input/redditnews/upload_DJIA_table_preprocessed.csv')

In [3]:
# Calculate the mean sentiment score for each stock for each day
df_finbert = df_finbert.groupby('Date') \
       .agg({'sentiment_FinBert_positive':'mean',
            'sentiment_FinBert_negative':'mean',
            'sentiment_FinBert_neutral':'mean'}) \
       .rename(columns={'sentiment_FinBert_positive':'sentiment_FinBert_positive',
                        'sentiment_FinBert_negative':'sentiment_FinBert_negative',
                        'sentiment_FinBert_neutral':'sentiment_FinBert_neutral'}) \
       .reset_index()

# Calculate the mean sentiment score for each stock for each day
df_vader = df_vader.groupby('Date') \
       .agg({'sentiment_vader':'mean'}) \
       .rename(columns={'sentiment_vader':'sentiment_vader'}) \
       .reset_index()

In [4]:
df_finbert = df_finbert.merge(df_stock[['Date', 'returns']], on='Date')
df_vader = df_vader.merge(df_stock[['Date', 'returns']], on='Date')

df_finbert['Date'] = pd.to_datetime(df_finbert['Date'])
df_vader['Date'] = pd.to_datetime(df_vader['Date'])

# set the index
df_finbert.set_index('Date', inplace=True)
df_vader.set_index('Date', inplace=True)

In [6]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
import h5py


# Define a function to prepare the data for the LSTM
def prepare_data(data, timesteps):
    X = []
    y = []
    
    for i in range(timesteps, len(data)):
        X.append(data[i-timesteps:i])
        y.append(data[i, -1])
    
    return np.array(X), np.array(y)

def run_LSTM(df, timesteps = 60):
    # Calculate the index to split the data
    split_idx = int(len(df) * 0.8)

    # Split the data into training and testing sets
    train = df.iloc[:split_idx]
    test = df.iloc[split_idx:]
    
    # Prepare the training and testing data
    X_train, y_train = prepare_data(train.values, timesteps)
    X_test, y_test = prepare_data(test.values, timesteps)

    # Get the number of features from the training data
    num_features = X_train.shape[-1]

    # Reshape the data to the format required by the LSTM
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], num_features))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], num_features))
    
    # Initialize the model
    model = Sequential()

    # Add the LSTM layer
    model.add(LSTM(units=50, return_sequences=False, input_shape=(X_train.shape[1], num_features)))

    # Add the output layer
    model.add(Dense(units=1))

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    # Define the callbacks
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=10),
        ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss'),
        TensorBoard(log_dir='./logs')
    ]
    
    # Train the model
    model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=callbacks)

    # Make predictions
    y_pred = model.predict(X_test)
    
    return y_pred,y_test


In [7]:
y_pred_finbert, y_test_finbert = run_LSTM(df_finbert)
y_pred_vader, y_test_vader = run_LSTM(df_vader)

Epoch 1/100
39/39 [==============================] - 8s 20ms/step - loss: 5.0446e-04 - val_loss: 7.1485e-05
Epoch 2/100
39/39 [==============================] - 0s 8ms/step - loss: 1.7343e-04 - val_loss: 4.7185e-05
Epoch 3/100
39/39 [==============================] - 0s 7ms/step - loss: 1.6342e-04 - val_loss: 5.5559e-05
Epoch 4/100
39/39 [==============================] - 0s 8ms/step - loss: 1.7195e-04 - val_loss: 5.1173e-05
Epoch 5/100
39/39 [==============================] - 0s 7ms/step - loss: 1.6305e-04 - val_loss: 5.6451e-05
Epoch 6/100
39/39 [==============================] - 0s 7ms/step - loss: 1.7434e-04 - val_loss: 9.5213e-05
Epoch 7/100
39/39 [==============================] - 0s 8ms/step - loss: 1.7132e-04 - val_loss: 5.6760e-05
Epoch 8/100
39/39 [==============================] - 0s 8ms/step - loss: 1.7514e-04 - val_loss: 4.5116e-05
Epoch 9/100
39/39 [==============================] - 0s 9ms/step - loss: 1.6496e-04 - val_loss: 4.4808e-05
Epoch 10/100
39/39 [================

In [26]:
# Convert them into pandas dataframes
y_pred_finbert_df = pd.DataFrame(y_pred_finbert, columns=['y_pred_FinBert'])
y_test_finbert_df = pd.DataFrame(y_test_finbert, columns=['y_test_FinBert'])
y_pred_vader_df = pd.DataFrame(y_pred_vader, columns=['y_pred_Vader'])
y_test_vader_df = pd.DataFrame(y_test_vader, columns=['y_test_Vader'])

# Combine them into a single dataframe
results = pd.concat([y_pred_finbert_df, y_test_finbert_df,y_pred_vader_df, y_test_vader_df], axis=1)

In [28]:
results
results.to_csv('RedditNews_Results.csv', index=False)